## Exporting Data
You can export images, map tiles, tables and video from Earth Engine. The exports can be sent to your Google Drive account, to Google Cloud Storage or to a new Earth Engine asset.

### Exporting images
You can export images from Earth Engine in GeoTIFF or TFRecord format. See Configuration Parameters for more output options. To export an image to your Drive account, use ee.batch.Export.image.toDrive(). For example, to export portions of a Landsat image, define a region to export, then call ee.batch.Export.image.toDrive():



In [2]:
# Initializing display, earthengine, and ipyleaflet
from IPython.display import Image
%matplotlib inline

import ee
ee.Initialize()

In [3]:
# Load a landsat image and select three bands.
landsat = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_123032_20140515').select(['B4', 'B3', 'B2'])

#// Create a geometry representing an export region.
geometry = ee.Geometry.Rectangle([116.2621, 39.8412, 116.4849, 40.01236])

task = ee.batch.Export.image.toDrive(image=landsat.clip(geometry),
                                           description="LC08_123032_20140515_TOA",
                                           folder="GEEexport",
                                           region=geometry['coordinates'],
                                           crs= landsat.projection().crs().getInfo(),
                                           scale=landsat.projection().nominalScale().getInfo(),
                                           fileFormat='GeoTIFF',
                                           skipEmptyTiles=True)
task.start()

## Exporting tables and vector data
You can export a FeatureCollection as CSV, SHP (shapefile), GeoJSON, KML, KMZ or TFRecord using Export.table. The FeatureCollection may represent vectors or simply a table of data. In the latter case, the features in the collection will have null geometry.

Note some additional constraints when working with some file formats, including:

KML: A FeatureCollection exported to a KML file will have all the geometries transformed to unprojected (WGS84) coordinates.
SHP: A FeatureCollection exported to a Shapefile must contain features with the same geometry type and projection and must fit within the Shapefile size limits. Column names are truncated to 10 characters or fewer, and this must not create duplicate column names.

In [4]:
#// Make a collection of points.
features = ee.FeatureCollection([
  ee.Feature(ee.Geometry.Point(30.41, 59.933)).set('name','Voronoi'),
  ee.Feature(ee.Geometry.Point(-73.96, 40.781)).set('name','Thiessen'),
  ee.Feature(ee.Geometry.Point(6.4806, 50.8012)).set('name','Dirichlet')
])

#// Export the FeatureCollection to a KML file.
task = ee.batch.Export.table.toDrive(
  collection= features,
  description='vectorsToDriveExample',
  folder="GEEexport",
  fileFormat= 'KML'
);


task.start()

Note that the output format is specified as KML to handle geographic data (SHP would also be appropriate for exporting a table with geometry). To export just a table of data, without any geographic information, export features with null geometry in CSV format. The following demonstrates using ee.batch.Export.table.toDrive() to get the results of a potentially long running reduction:



In [5]:
# // Load a Landsat TOA image.
image = ee.Image('LANDSAT/LC08/C01/T1_TOA/LC08_044034_20140318');
# // Create an arbitrary rectangle.
region = ee.Geometry.Rectangle([-122.2806, 37.1209, -122.0554, 37.2413])

# // Get a dictionary of means in the region.
means = image.reduceRegion(ee.Reducer.mean(),region,  maxPixels= 1e9)
print(means.getInfo())

{'B1': 0.10156726419256697, 'B10': 288.3898316688731, 'B11': 287.4496961587173, 'B2': 0.07771978773589312, 'B3': 0.05530074222374056, 'B4': 0.03670883505442356, 'B5': 0.22139764553479283, 'B6': 0.07888235294207073, 'B7': 0.035089536963059234, 'B8': 0.047100656968120116, 'B9': 0.00156092084691453, 'BQA': 2720.11945357173}


In [6]:
# // Make a feature without geometry and set the properties to the dictionary of means.
feature = ee.Feature(None, means)

# // Wrap the Feature in a FeatureCollection for export.
featureCollection = ee.FeatureCollection([feature])

# // Export the FeatureCollection.
task = ee.batch.Export.table.toDrive(
  collection=featureCollection,
  description= 'exportTableExample',
  folder="GEEexport",
  fileFormat='CSV'
)

task.start()